# DART 전자공시 API Crawler

##0. Prerequsite

In [0]:
# install Python packages
!pip3 install lxml html5lib aiohttp arsenic selenium

# install chromium-browser & chromedriver
# binary_location: /usr/lib/chromium-browser/chromium-browser
# executable_path: /usr/lib/chromium-browser/chromedriver
!apt-get update && apt-get -y upgrade
!apt-get install chromium-chromedriver
!apt-get -f install
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!cp /usr/lib/chromium-browser/chromium-browser /usr/bin/chrome
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

##1. 법인목록 다운로드

In [0]:
config = {
    "host": "http://kind.krx.co.kr/corpgeneral/corpList.do",
    "method": "download",
    "marketType": {
        "rAll": "",
        "rWertpapier": "stockMkt", 
        "rKosdaq": "kosdaqMkt", 
        "rKonex": "konexMkt"
    },
    "searchType": {
        "상장법인": "13",
        "관리종목": "01",
        "불성실공시법인": "05",
        "자산2조법인": "07",
        "외국법인": "99",
        "(코스닥) 우량기업부": "181",
        "(코스닥) 벤처기업부": "182",
        "(코스닥) 중견기업부": "183",
        "(코스닥) 기술성장기업부": "184",
        "KRX100": "11",
        "KOSPI200": "06",
        "STAR30": "09",
        "PREMIER": "10"
    },
    "industry": {
        "전체":"",
        "농업, 임업 및 어업": "01",
        "광업": "02",
        "제조업": {
            "전체":"03",
            "식료품": "0310",
            "음료": "0311",
            "담배": "0312",
            "섬유제품; 의복제외": "0313",
            "의복, 의복액세서리 및 모피제품": "0314",
            "가죽, 가방 및 신발": "0315",
            "목재 및 나무제품;가구제외": "0316",
            "펄프, 종이 및 종이제품": "0317",
            "인쇄 및 기록매체 복제업": "0318",
            "코크스, 연탄 및 석유정제품": "0319",
            "화학물질 및 화학제품;의약품 제외": "0320",
            "의료용 물질 및 의약품": "0321",
            "고무 및 플라스틱제품": "0322",
            "비금속 광물제품": "0323",
            "1차 금속": "0324",
            "금속가공제품;기계 및 가구 제외": "0325",
            "전자부품, 컴퓨터, 영상, 음향 및 통신장비": "0326",
            "의료, 정밀, 광학기기 및 시계": "0327",
            "전기장비": "0328",
            "기타 기계 및 장비": "0329",
            "자동차 및 트레일러": "0330",
            "기타 운송장비": "0331",
            "가구": "0332",
            "기타 제품": "0333",
            "산업용 기계 및 장비": "0334"
        },
        "전기, 가스, 증기 및 공기조절 공급업": "04",
        "수도, 하수 및 폐기물 처리, 원료 재생업": "05",
        "건설업": "06",
        "도매 및 소매업": "07",
        "운수 및 창고업": "08",
        "숙박 및 음식점업": "09",
        "정보통신업": "10",
        "금융 및 보험업": "11",
        "부동산업": "12",
        "전문, 과학 및 기술 서비스업": "13",
        "사업시설 관리, 사업지원 및 임대 서비스업": "14",
        "공공행정, 국방 및 사회보장 행정": "15",
        "교육 서비스업": "16",
        "보건업 및 사회복지 서비스업": "17",
        "예술, 스포츠 및 여가관련 서비스업": "18",
        "협회 및 단체, 수리 및 기타 개인 서비스업": "19",
        "가구 내 고용활동 및 달리 분류되지 않은 자가소비 생산활동": "20",
        "국제 및 외국기관": "21",
    },
    "fiscalYearEnd": {
        "전체": "all",
        "01월": "01",
        "02월": "02",
        "03월": "03",
        "04월": "04",
        "05월": "05",
        "06월": "06",
        "07월": "07",
        "08월": "08",
        "09월": "09",
        "10월": "10",
        "11월": "11",
        "12월": "12"
    },
    "location": {
        "전체": "all",
        "강원도": "01",
        "경기도": "02",
        "경상남도": "03",
        "경상북도": "04",
        "광주광역시": "05",
        "대구광역시": "06",
        "대전광역시": "07",
        "부산광역시": "08",
        "서울특별시": "09",
        "세종특별자치시": "10",
        "울산광역시": "11",
        "인천광역시": "12",
        "전라남도": "13",
        "전라북도": "14",
        "제주특별자치도": "15",
        "충청남도": "16",
        "충청북도": "17",
        "외국법인": "18"
    }
}

In [0]:
import aiohttp
from aiohttp import ClientSession
import asyncio
from bs4 import BeautifulSoup
import pandas as pd
import time


class AsyncKrxCorpListCrawler():
    def __init__(self, host, method, searchType, industry, fiscalYearEnd, location, marketType, max_concurrency=10):
        self.host = host
        self.method = method
        self.searchType = searchType
        self.industry = industry
        self.fiscalYearEnd = fiscalYearEnd
        self.location = location
        self.marketType = marketType
        self.marketTypeList = (marketType[key] for key in self.marketType.keys() if key != "rAll")
        self.boundedSempahore = asyncio.BoundedSemaphore(max_concurrency)

    async def fetchHtmlResponse(self, url):
        async with aiohttp.ClientSession() as session:
            try:
                async with session.get(url) as response:
                    if response.status == 200:
                        html = await response.content.read()
                        return html
                    else:
                        return {'error': response.status, 'html': ''}
            except Exception as err:
                return {'error': err, 'html': ''}
                    
    def parseHtmlTable(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')[0]
        return table

    async def convertHtmlTableToDataFrame(self, marketType):
        try:
            with (await self.boundedSempahore):
                uri = f'{self.host}?method={self.method}&searchType={self.searchType}&industry={self.industry}&fiscalYearEnd={self.fiscalYearEnd}&location={self.location}&marketType={marketType}'
                req = await self.fetchHtmlResponse(uri)
                table = self.parseHtmlTable(req)
                df = pd.read_html(str(table), header=0)[0]
                df['marketType'] = marketType
                df['종목코드'] = df['종목코드'].astype(str).str.zfill(6)
            return df
        
        except Exception as err:
            return {'error': err, 'table': pd.DataFrame()}

    async def concatKRXCorpList(self):
        KRXCorpDataFrameList = [self.convertHtmlTableToDataFrame(marketType) for marketType in self.marketTypeList]
        KRXCorpDataFrameIter = asyncio.as_completed(KRXCorpDataFrameList)

        df = pd.DataFrame()
        for future in KRXCorpDataFrameIter:
            df = pd.concat([df, await future])

        return df

host = config['host']
method = config['method']
searchType = config['searchType']['상장법인']
industry = config['industry']['전체']
fiscalYearEnd = config['fiscalYearEnd']['전체']
location = config['location']['전체']
marketType = config['marketType']

t_start = time.time()

asyncKRXCorpListCrawler = AsyncKrxCorpListCrawler(host, method, searchType, industry, fiscalYearEnd, location, marketType)
loop = asyncio.get_event_loop()
task = asyncio.ensure_future(asyncKRXCorpListCrawler.concatKRXCorpList())
KRXCorpList = loop.run_until_complete(task)

t_end = time.time()

m, s = divmod(t_end-t_start, 60)
h, m = divmod(m, 60)

print('Duration : %02d:%02d:%02d' % (h, m, s))

Duration : 00:00:04


In [0]:
from bs4 import BeautifulSoup as bs
from functools import reduce
import pandas as pd
import requests
import time

# 현재 한국거래서(KRX)에서 '법인목록' 다운로드 시 '시장구분(marketType) 필드가 없다.'
# 따라서 '시장구분' 필드를 추가하여 법인목록 데이터를 가져올 수 있도록 한다.
host = config['host']
method = config['method']
searchType = config['searchType']['상장법인']
industry = config['industry']['전체']
fiscalYearEnd = config['fiscalYearEnd']['전체']
location = config['location']['전체']

markets = {key:value for key, value in config['marketType'].items() if key != "rAll"}
url = f'{host}?method={method}&searchType={searchType}&industry={industry}&fiscalYearEnd={fiscalYearEnd}&location={location}'

def generateCorpList(markets):
    for key, value in markets.items():
        uri = f'{url}&marketType={value}'
        uriData = requests.get(uri)
        soup = bs(uriData.text, 'html.parser')
        table = soup.find_all('table')[0]
        df = pd.read_html(str(table), header=0)[0]
        df['marketType'] = value
        df['종목코드'] = df['종목코드'].astype(str).str.zfill(6)
        yield df

t_start = time.time()
corpList = reduce(lambda x,y: pd.concat([x,y]), generateCorpList(markets))
corpList = corpList.reset_index(drop=True)
t_end = time.time()

m, s = divmod(t_end-t_start, 60)
h, m = divmod(m, 60)

print('Duration : %02d:%02d:%02d' % (h, m, s))

Duration : 00:00:07


In [0]:
KRXCorpList.head()
# corpList.loc[corpList['종목코드'].astype(str).str.len() != 6]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,marketType
0,다이노나,086080,자연과학 및 공학 연구개발업,항체치료제,2018-05-08,12월,송형근,http://www.dinonainc.com,서울특별시,konexMkt
1,대동고려삼,178600,기타 식품 제조업,"인삼식품(홍삼진액, 홍삼정)",2013-12-20,06월,최 성 근,http://www.ddkorea.co.kr,충청남도,konexMkt
2,디피코,163430,"건축기술, 엔지니어링 및 관련 기술 서비스업","자동차 엔지니어링 서비스, 의료용 전동스쿠터 등",2018-05-16,12월,송신근,http://www.dpeco.com,경기도,konexMkt
3,라이프사이언스테크놀로지,285770,소프트웨어 개발 및 공급업,생체신호모듈 및 패치형 체온 측정시스템 등,2017-12-21,12월,김정환,http://www.lstgrp.com,서울특별시,konexMkt
4,로보쓰리,238500,"그외 기타 전문, 과학 및 기술 서비스업","모듈, 퍼스널 모빌리티기기",2016-04-20,12월,김준형,http://www.robo3.com,서울특별시,konexMkt


###2. DART > 사업보고서 내용(Hyperlink) 파싱

In [0]:
from time import time
import datetime, pytz
from operator import eq


import os
import re
import json
import lxml
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

configs = {
    "apikey": "apikey",
    "start_dt": 20170101,
    "end_dt": 20171231,
    "bsn_tp": "A001",
    "content": "II. 사업의 내용"
}
configs["crp_cd"] = '001040'

url = "http://dart.fss.or.kr/api/search.json?auth={apikey}&crp_cd={crp_cd}&start_dt={start_dt}&end_dt={end_dt}&bsn_tp={bsn_tp}".format(**configs)
print('url_search:', url)
rawData = requests.get(url).text
parsedData = json.loads(rawData)
print('err_code:', parsedData['err_code'])
parsedDF = pd.DataFrame(parsedData['list'])

for data in  parsedData['list']:
    rcpNo = data['rcp_no']
    url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcpNo}'.format(rcpNo=rcpNo)
    req = requests.get(url)
    html = req.text
    treeNodeJS = re.findall("TreeNode\(\{.*?\}\)\;", html, re.I|re.S)[1:] #목차 내용만 추출

    treeNodeJson = []
    for node in treeNodeJS:
        foam = {}
        foam["crp_cd"] = data["crp_cd"]
        foam["rpt_cnt"] = re.search('text\:.*?\,', node).group()[7:-2]
        foam["id"] = re.search('id\:.*?\,', node).group()[5:-2]
        foam["cls"] = re.search('cls\:.*?\,', node).group()[6:-2]
        listenersKey = ["rcpNo", "dcmNo", "eleId", "offset", "length", "dtd"]
        listenersValue = re.search('\{viewDoc\(.*?\}', node, re.I|re.S).group()[9:-3]
        listenersValue = re.sub('[\s\']', '', listenersValue).split(',')
        listeners = {key:value for key, value in zip(listenersKey, listenersValue)}
        foam = {**foam, **listeners}
        hyperlink = 'http://dart.fss.or.kr/report/viewer.do?rcpNo={rcpNo}&dcmNo={dcmNo}&eleId={eleId}&offset={offset}&length={length}&dtd={dtd}'.format(**listeners)
        foam["hyperlink"] = hyperlink
        treeNodeJson.append(foam)

        treeNodeDF = pd.DataFrame(treeNodeJson)
        rpt_cnt = ["II. 사업의 내용"]
        rpt = pd.merge(parsedDF, treeNodeDF.loc[treeNodeDF['rpt_cnt'].isin(rpt_cnt)], how='inner', left_on=['crp_cd','rcp_no'], right_on=['crp_cd','rcpNo'])
